In [1]:
import os
import time
import numpy as np
from time import perf_counter
from IPython.display import clear_output

import brainflow
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds, BrainFlowError

from tqdm.notebook import tqdm
import datetime



from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import itertools
from sklearn.utils import shuffle
from scipy import signal


%matplotlib qt

%load_ext autoreload
%autoreload 2


from utils.svm import preProcess_1, DCFilter
from utils.visualize import Vis, VisAction, showMe, stat

C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
res_dir = 'C:/resources/EMG/'+datetime.datetime.now().strftime("%m_%d")+'/'
try:
    os.mkdir(res_dir)
except:
    ...

In [3]:
####      INIT BOARD        #######
BoardShim.enable_dev_board_logger()
params = BrainFlowInputParams()
board = BoardShim(BoardIds.MINDROVE_WIFI_BOARD, params)


try:
    board.stop_stream()
    board.release_session()
except:
    ...
    
    
board.prepare_session()
sample_rate = board.get_sampling_rate(16)
n_channels  = 6

print("Device ready (sampling rate: {}hz)".format(sample_rate))

Device ready (sampling rate: 500hz)


In [4]:
length_of_signal = 1 #seconds
num_points = int(sample_rate * length_of_signal)


record_length = int(sample_rate * length_of_signal*2)

In [7]:
def GenerateOrder(n_classes, n_samples_per_class = 1):
    lists = []
    for i in range(n_classes):
        tmp = np.empty([n_samples_per_class])
        tmp.fill(i)
        lists.append(tmp)
    order = np.vstack(lists).ravel().astype(np.int32)
    np.random.shuffle(order)
    return order


def CollectData(classes = ['Eyebrow','Smile', 'Chew'],n_samples_per_class = 1):
    classes = ['Rest'] + classes
    results = [[] for i in range(len(classes))]
    tasks = GenerateOrder(len(classes),n_samples_per_class)
  
    board.start_stream(500*60) 
    print("Setting up...")
    time.sleep(10)
    for i, task in enumerate(tasks):
        #clear_output()
        print("Stand By! ({}/{})".format(i+1,len(tasks)))

        time.sleep(1)
        print("Perform |{:^10s}|".format(classes[tasks[i]]))
        time.sleep(2)
        data = board.get_current_board_data(sample_rate*60) #1min
       
        data = DCFilter(data)
        data = data[:6,-record_length:] #keep the data of the eeg channels only, and remove data over the trial length
        _, _, std = stat(data)
        if std == 0:
            print("[ERROR] No data collected!")
            break
        results[task].append(data)

    return results, classes
        
results, classes = CollectData(n_samples_per_class=30)  #5min

BrainFlowError: STREAM_ALREADY_RUN_ERROR:8 unable to start streaming session

In [9]:
a = np.array([0,1,2,3,4,5]).reshape(-1,1)
print(a)
a = a[-2:]
print(a)

[[0]
 [1]
 [2]
 [3]
 [4]
 [5]]
[[4]
 [5]]


In [6]:
for i, category in enumerate(results):
    print(classes[i])
    for command in category:
        command = DCFilter(command)
        VisAction(command, std_threshold)
        print(stat(command))
        time.sleep(1)

Rest
-200
(-241.0, 528.0, 140.0)
-200
(-339.0, 694.0, 204.0)
-200
(-684.0, 635.0, 243.0)
Blink
-200
(-2071.0, 7810.0, 3461.0)
-200
(-1796.0, 6505.0, 2377.0)
-200
(-593.0, 1642.0, 686.0)
Smile
-200
(-3181.0, 1631.0, 1403.0)


KeyboardInterrupt: 

In [16]:
showMe(results[2][0])

In [6]:
#Save results to file
i = 0
for result in tqdm(results):
    result= np.asarray(result)
    np.save(res_dir+classes[i],result)
    i+=1

  0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
board.stop_stream()
board.release_session()